# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 51 new papers today
          16 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/15 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.12267


extracting tarball to tmp_2302.12267...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.12267/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: 'table_mrt' from 'tmp_2302.12267/table_mrt.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:312: LatexWarning: Latex injecting: 'table_fit' from 'tmp_2302.12267/table_fit.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_dec

...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.12788


extracting tarball to tmp_2302.12788...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.12805


extracting tarball to tmp_2302.12805...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12805-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12805) | **JADES: Discovery of extremely high equivalent width Lyman-alpha emission  from a faint galaxy within an ionized bubble at $z=7.3$**  |
|| Aayush Saxena, et al. -- incl., <mark>Anna De Graaff</mark>, <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2023-02-27*|
|*Comments*| *12 pages, 7 figures, submitted to Astronomy & Astrophysics, comments welcome!*|
|**Abstract**| We report the discovery of a remarkable Ly$\alpha$ emitting galaxy at $z=7.278$, JADES-GS+53.16746-27.7720 (shortened to JADES-GS-z7-LA), with EW$_0$(Ly$\alpha$) $\approx400 \pm 90$A and UV magnitude $-16.7$. The spectroscopic redshift is confirmed via rest-frame optical lines [O II], H$\beta$ and [O III] in its JWST/NIRSpec Micro-Shutter Assembly (MSA) spectrum. The Ly$\alpha$ line is detected in both lower resolution ($R\sim100$) PRISM as well as medium resolution ($R\sim1000$) G140M grating spectra. The Ly$\alpha$ FWHM in the grating is $\approx360$ km s$^{-1}$ and the line peaks within $120$ km s$^{-1}$ of the systemic redshift, indicative of very little neutral gas or dust within the galaxy. We estimate the Ly$\alpha$ escape fraction to be $\sim100\%$. JADES-GS-z7-LA has a [O III]/[O II] ratio (O32) of $8.8 \pm 1.1$ and ([O III]+[O II])/H$\beta$ ratio (R23) of $9.6\pm2.2$, consistent with low metallicity and high ionization parameters. Deep NIRCam imaging also revealed a close companion source (separated by $0.23''$), which exhibits similar photometry to that of JADES-GS-z7-LA, with a photometric excess in the F410M NIRCam image consistent with [O III]+H$\beta$ emission at the same redshift. The spectral energy distribution of JADES-GS-z7-LA indicates a `bursty' star-formation history, with a low stellar mass of $10^{7.15}$ M$_\odot$. The only explanation of the high EW Ly$\alpha$ emission seen in JADES-GS-z7-LA is if it resides in an ionized bubble with radius $\gtrsim 3$ pMpc. Owing to the faint nature of JADES-GS-z7-LA, we show that it is incapable of single-handedly ionizing a bubble large enough. Therefore, we suggest that JADES-GS-z7-LA (and possibly the companion source) may be a part of a larger overdensity, presenting direct evidence of overlapping ionized bubbles at $z>7$, allowing us to study the process of reionization across both small and large scales. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12269-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12269) | **The origin of power-law spectra in relativistic magnetic reconnection**  |
|| <mark>Hao Zhang</mark>, Lorenzo Sironi, Dimitrios Giannios, Maria Petropoulou |
|*Appeared on*| *2023-02-27*|
|*Comments*| *8 pages, 8 figures, submitted*|
|**Abstract**| Magnetic reconnection is often invoked as a source of high-energy particles, and in relativistic astrophysical systems it is regarded as a prime candidate for powering fast and bright flares. We present a novel analytical model - supported and benchmarked with large-scale three-dimensional particle-in-cell simulations - that elucidates the physics governing the generation of power-law energy spectra in relativistic reconnection. Particles with Lorentz factor $\gamma\gtrsim 3\sigma$ (here, $\sigma$ is the magnetization) gain most of their energy in the inflow region, while meandering between the two sides of the reconnection layer. Their acceleration time is $t_{\rm acc}\sim \gamma \,\eta_{\rm rec}^{-1}\omega_{\rm c}^{-1}\simeq 20\,\gamma\,\omega_{\rm c}^{-1}$, where $\eta_{\rm rec}\simeq0.06$ is the inflow speed in units of the speed of light and $\omega_{\rm c}=eB_0/mc$ is the gyrofrequency in the upstream magnetic field. They leave the region of active energization after $t_{\rm esc}$, when they get captured by one of the outflowing flux ropes of reconnected plasma. We directly measure $t_{\rm esc}$ in our simulations and find that $t_{\rm esc}\sim t_{\rm acc}$ for $\sigma\gtrsim {\rm few}$. This leads to a universal (i.e., $\sigma$-independent) power-law spectrum $dN_{\rm free}/d\gamma\propto \gamma^{-1}$ for the particles undergoing active acceleration, and $dN/d\gamma\propto \gamma^{-2}$ for the overall particle population. Our results help shedding light on the ubiquitous presence of power-law particle and photon spectra in astrophysical non-thermal sources. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12325-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12325) | **The Intrinsic Alignment of Red Galaxies in DES Y1 redMaPPer Galaxy  Clusters**  |
|| C. Zhou, et al. -- incl., <mark>Y. Zhang</mark> |
|*Appeared on*| *2023-02-27*|
|*Comments*| *14 pages, 13 figures*|
|**Abstract**| Clusters of galaxies are sensitive to the most nonlinear peaks in the cosmic density field. The weak gravitational lensing of background galaxies by clusters can allow us to infer their masses. However, galaxies associated with the local environment of the cluster can also be intrinsically aligned due to the local tidal gradient, contaminating any cosmology derived from the lensing signal. We measure this intrinsic alignment in Dark Energy Survey (DES) Year 1 redMaPPer clusters. We find evidence of a non-zero mean radial alignment of galaxies within clusters between redshift 0.1-0.7. We find a significant systematic in the measured ellipticities of cluster satellite galaxies that we attribute to the central galaxy flux and other intracluster light. We attempt to correct this signal, and fit a simple model for intrinsic alignment amplitude ($A_{\textrm{IA}}$) to the measurement, finding $A_{\textrm{IA}}=0.15\pm 0.04$, when excluding data near the edge of the cluster. We find a significantly stronger alignment of the central galaxy with the cluster dark matter halo at low redshift and with higher richness and central galaxy absolute magnitude (proxies for cluster mass). This is an important demonstration of the ability of large photometric data sets like DES to provide direct constraints on the intrinsic alignment of galaxies within clusters. These measurements can inform improvements to small-scale modeling and simulation of the intrinsic alignment of galaxies to help improve the separation of the intrinsic alignment signal in weak lensing studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12723-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12723) | **JWST Observations of the Enigmatic Y Dwarf WISE 1828+2650: I. Limits to  a Binary Companion**  |
|| Matthew De Furio, et al. -- incl., <mark>Michael R. Meyer</mark> |
|*Appeared on*| *2023-02-27*|
|*Comments*| *15 pages, 9 figures, Accepted by ApJ on Feb. 21 2023*|
|**Abstract**| The Y-dwarf WISE 1828+2650 is one of the coldest known Brown Dwarfs with an effective temperature of $\sim$300 K. Located at a distance of just 10 pc, previous model-based estimates suggest WISE1828+2650 has a mass of $\sim$5-10 Mj, making it a valuable laboratory for understanding the formation, evolution and physical characteristics of gas giant planets. However, previous photometry and spectroscopy have presented a puzzle with the near-impossibility of simultaneously fitting both the short (0.9-2.0 microns) and long wavelength (3-5 microns) data. A potential solution to this problem has been the suggestion that WISE 1828+2650 is a binary system whose composite spectrum might provide a better match to the data. Alternatively, new models being developed to fit JWST/NIRSpec and MIRI spectroscopy might provide new insights. This article describes JWST/NIRCam observations of WISE 1828+2650 in 6 filters to address the binarity question and to provide new photometry to be used in model fitting. We also report Adaptive Optics imaging with the Keck 10 m telescope. We find no evidence for multiplicity for a companion beyond 0.5 AU with either JWST or Keck. Companion articles will present low and high resolution spectra of WISE 1828+2650 obtained with both NIRSpec and MIRI. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12737-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12737) | **A Model for Nonthermal Particle Acceleration in Relativistic Magnetic  Reconnection**  |
|| <mark>Xiaocan Li</mark>, <mark>Fan Guo</mark>, Yi-Hsin Liu, <mark>Hui Li</mark> |
|*Appeared on*| *2023-02-27*|
|*Comments*| *6 pages, 5 figures, submitted*|
|**Abstract**| The past decade has seen an outstanding development of nonthermal particle acceleration in magnetic reconnection in magnetically-dominated systems, with clear signatures of power-law energy distributions as a common outcome of first-principles kinetic simulations. Here we propose a semi-analytical model for systematically investigating nonthermal particle acceleration in reconnection. We show particle energy distributions are well determined by particle injection, acceleration, and escape processes. Using a series of kinetic simulations, we accurately evaluate the energy- and time-dependent model coefficients. The resulting spectral characteristics, including the spectral index and lower and upper bounds of the power-law distribution, agree well with the simulation results. Finally, we apply the model to predict the power-law indices and break energies in astrophysical reconnection systems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12788-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12788) | **The interplay between accretion, galaxy downsizing and the formation of  box/peanut bulges in TNG50**  |
|| Stuart Robert Anderson, et al. -- incl., <mark>Joseph Caruana</mark> |
|*Appeared on*| *2023-02-27*|
|*Comments*| *20 pages, 24 figures, 5 tables. Submitted to MNRAS*|
|**Abstract**| From the TNG50 cosmological simulation we build a sample of 191 well-resolved barred galaxies with a stellar mass $\logMstar > 10$ at $z=0$. We search for box/peanut bulges (BPs) in this sample, finding them in 55\% of cases. We compute $\fbp$, the BP probability for barred galaxies as a function of $\Mstar$, and find that this rises to a plateau, as found in observations of nearby galaxies. The transition mass where $\fbp$ reaches half the plateau value is $\logMstar = 10.14$, consistent with the observational value within measurement errors. We show that this transition in $\fbp$ can be attributed to the youth of the bars at low $\Mstar$, which is a consequence of downsizing of galaxies. Young bars, being generally shorter and weaker, have not yet had time to form BPs. At high mass, while we find a plateau, the value is at $\sim 60\%$ whereas observations saturate at $100\%$. We attribute this difference to excessive heating in TNG50, due to merger activity and to numerical resolution effects. BPs in TNG50 tend to occur in galaxies with more quiescent merger histories. As a result, the main driver of whether a bar hosts a BP in TNG50 is not the galaxy mass, but how long and strong the bar is. Separating the BP sample into those that have visibly buckled and those that have not, we find that fully half of BP galaxies show clear signs of buckling, despite the excessive heating and limited vertical resolution of TNG50. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12753-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12753) | **Thermal history of the Erg Chech 002 parent body: Early accretion and  early differentiation of a small asteroid**  |
|| <mark>Wladimir Neumann</mark>, et al. |
|*Appeared on*| *2023-02-27*|
|*Comments*| **|
|**Abstract**| The history of accretion and differentiation processes in the planetesimals is provided by various groups of meteorites. Sampling different parent body layers, they reveal the circumstances of the metal-silicate segregation and the internal structures of the protoplanets. The ungrouped achondrite Erg Chech 002 (EC 002) added to the suite of samples from primitive igneous crusts. Here we present models that utilize thermo-chronological data for EC 002 and fit the accretion time and size of its parent body to these data. The U-corrected Pb-Pb, Al-Mg, and Ar-Ar ages used imply a best-fit planetesimal with a radius of 20-30 km that formed at 0.1 Ma after CAIs. Its interior melted early and differentiated by 0.5 Ma, allowing core and mantle formation with a transient lower mantle magma ocean, and a melt fraction of <25 % at the meteorite layering depth. EC 002 formed from this melt at a depth of 0.8 km in a partially differentiated region covered by an undifferentiated crust. By simulating collisions with impactors of different sizes and velocities, we analyzed the minimum ejection conditions of EC 002 from its original parent body and the surface composition of the impact site. The magma ocean region distinct from the layering depth of EC 002 implyes that it was not involved in the EC 002 genesis. Our models estimate closure temperatures for the Al-Mg ages as 1060 K to 1200 K. A fast parent body cooling attributes the late Ar-Ar age to a local reheating by another, late impact. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.12267-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.12267) | **Star Formation Laws and Efficiencies across 80 Nearby Galaxies**  |
|| Jiayi Sun, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Annie Hughes</mark>, <mark>Kathryn Kreckel</mark>, <mark>Frank Bigiel</mark>, <mark>Lukas Neumann</mark> |
|*Appeared on*| *2023-02-27*|
|*Comments*| *10 pages main text + 2 appendices. ApJL in press. Data products available at this https URL . Slides summarizing key results can be found at this https URL*|
|**Abstract**| We measure empirical relationships between the local star formation rate (SFR) and properties of the star-forming molecular gas on 1.5 kpc scales across 80 nearby galaxies. These relationships, commonly referred to as "star formation laws," aim at predicting the local SFR surface density from various combinations of molecular gas surface density, galactic orbital time, molecular cloud free-fall time, and the interstellar medium dynamical equilibrium pressure. Leveraging a multiwavelength database built for the PHANGS survey, we measure these quantities consistently across all galaxies and quantify systematic uncertainties stemming from choices of SFR calibrations and the CO-to-H$_2$ conversion factors. The star formation laws we examine show 0.3-0.4 dex of intrinsic scatter, among which the molecular Kennicutt-Schmidt relation shows a $\sim$10% larger scatter than the other three. The slope of this relation ranges $\beta\approx0.9{-}1.2$, implying that the molecular gas depletion time remains roughly constant across the environments probed in our sample. The other relations have shallower slopes ($\beta\approx0.6{-}1.0$), suggesting that the star formation efficiency (SFE) per orbital time, the SFE per free-fall time, and the pressure-to-SFR surface density ratio (i.e., the feedback yield) may vary systematically with local molecular gas and SFR surface densities. Last but not least, the shapes of the star formation laws depend sensitively on methodological choices. Different choices of SFR calibrations can introduce systematic uncertainties of at least 10-15% in the star formation law slopes and 0.15-0.25 dex in their normalization, while the CO-to-H$_2$ conversion factors can additionally produce uncertainties of 20-25% for the slope and 0.10-0.20 dex for the normalization. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.12805.md
    + _build/html/tmp_2302.12805/./figs/GS_LA7_Lya_R1000_vel.png
    + _build/html/tmp_2302.12805/./figs/GS_LA7_vel_comparison_v2.png
    + _build/html/tmp_2302.12805/./figs/LAE-companion-Bagpipes-fits-v0.7.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\lya}{Ly\textalpha}$
$\newcommand{\ha}{H\textalpha}$
$\newcommand{\hb}{H\textbeta}$
$\newcommand{\hg}{H\textgamma}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\xiion}{\xi_\mathrm{ion}}$
$\newcommand{\fesc}{f_\mathrm{esc}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\sfr}{M_\odot yr^{-1}}$
$\newcommand{\kms}{km s^{-1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\lya}{Ly\textalpha}$
$\newcommand{\ha}{H\textalpha}$
$\newcommand{\hb}{H\textbeta}$
$\newcommand{\hg}{H\textgamma}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\hii}{H \textsc{ii}}$
$\newcommand{\xiion}{\xi_\mathrm{ion}}$
$\newcommand{\fesc}{f_\mathrm{esc}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\sfr}{M_\odot yr^{-1}}$
$\newcommand{\kms}{km s^{-1}}$</div>



<div id="title">

# JADES: Discovery of extremely high equivalent width Lyman-alpha emission from a faint galaxy within an ionized bubble at $z=7.3$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.12805-b31b1b.svg)](https://arxiv.org/abs/2302.12805)<mark>Appeared on: 2023-02-27</mark> - _12 pages, 7 figures, submitted to Astronomy & Astrophysics, comments welcome!_

</div>
<div id="authors">

Aayush Saxena, et al. -- incl., <mark>Anna De Graaff</mark>, <mark>Hans-Walter Rix</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of a remarkable Ly $\alpha$ emitting galaxy at $z=7.278$ , JADES-GS+53.16746-27.7720 (shortened to JADES-GS-z7-LA), with EW $_0$ (Ly $\alpha$ ) $\approx400 \pm 90$ Å and UV magnitude $-16.7$ . The spectroscopic redshift is confirmed via rest-frame optical lines [ O ii ] , H $\beta$ and [ O iii ] in its JWST/NIRSpec Micro-Shutter Assembly (MSA) spectrum. The Ly $\alpha$ line is detected in both lower resolution ( $R\sim100$ ) PRISM as well as medium resolution ( $R\sim1000$ ) G140M grating spectra. The Ly $\alpha$ FWHM in the grating is $\approx360$ km s $^{-1}$ and the line peaks with [ O ii ] in $120$ km s $^{-1}$ of the systemic redshift, indicative of very little neutral gas or dust within the galaxy. We estimate the Ly $\alpha$ escape fraction to be $\sim100\%$ . JADES-GS-z7-LA has a [ O iii ] / [ O ii ] ratio (O32) of $8.8 \pm 1.1$ and ( [ O iii ] + [ O ii ] )/H $\beta$ ratio (R23) of $9.6\pm2.2$ , consistent with low metallicity and high ionization parameters. Deep NIRCam imaging also revealed a close companion source (separated by $0.23"$ ), which exhibits similar photometry to that of JADES-GS-z7-LA, with a photometric excess in the F410M NIRCam image consistent with [ O iii ] +H $\beta$ emission at the same redshift. The spectral energy distribution of JADES-GS-z7-LA indicates a `bursty' star-formation history, with a low stellar mass of $10^{7.15}$ M $_\odot$ . The only explanation of the high EW Ly $\alpha$ emission seen in JADES-GS-z7-LA is if it resides in an ionized bubble with radius $\gtrsim 3$ pMpc. Owing to the faint nature of JADES-GS-z7-LA, we show that it is incapable of single-handedly ionizing a bubble large enough. Therefore, we suggest that JADES-GS-z7-LA (and possibly the companion source) may be a part of a larger overdensity, presenting direct evidence of overlapping ionized bubbles at $z>7$ , allowing us to study the process of reionization across both small and large scales.

</div>

<div id="div_fig1">

<img src="tmp_2302.12805/./figs/GS_LA7_Lya_R1000_vel.png" alt="Fig1" width="100%"/>

**Figure 1. -** Velocity shift of the $\lya$ emission line in the G140M NIRSpec spectrum with a resolution of $R\sim1000$ compared to the systemic redshift ($\Delta v=0$\kms) traced by the $\oiii$ and $\hb$ lines. The emission line appears to be symmetric, centred very close to the systemic redshift with an offset of $120\pm80$\kms, with flux bluewards of the peak. The observed line profile is indicative of significant $\lya$ photon escape thanks to virtually no neutral gas along the line-of-sight in the galaxy. (*fig:R1000_spec*)

</div>
<div id="div_fig2">

<img src="tmp_2302.12805/./figs/GS_LA7_vel_comparison_v2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Distribution of $\lya$ EW and its velocity offset from the systemic redshift for JADES-GS-z7-LA (star), compared with GN-z11 \citep{bun23}(diamond), $z>7$ LAEs identified from CEERS by \citet{tan23}(circles) and other known $z>7$ LAEs from the \citet{end22c} compilation (squares), which include sources from the ALMA REBELS survey amongst others such as B14-65666 at $z=7.15$\citep{fur16, has19}, EGS-zs8-1 at $z=7.72$\citep{oes15, sta17}, COS-zs7-1 at $z=7.15$\citep{pen16, lap17, sta17} and BDF-3299 at $z=7.11$\citep{van11, mai15, car17}. We also include the $z=7.68$ galaxy Y002 \citep{val22}. JADES-GS-z7-LA lines up well with the observed strong anti-correlation between the strength of the $\lya$ emission line and its velocity offset compared to systemic redshift, and its faint UV magnitude and high $\lya$ EW compared to other $z>7$ LAEs stand out. (*fig:vel_comp*)

</div>
<div id="div_fig3">

<img src="tmp_2302.12805/./figs/LAE-companion-Bagpipes-fits-v0.7.png" alt="Fig3" width="100%"/>

**Figure 3. -** Best-fitting SEDs of JADES-GS-z7-LA and its companion source, fixing the redshift to be at $z=7.278$ for both objects. We find that a good fit is obtained for the companion source at the same redshift as JADES-GS-z7-LA, which in particular can explain the photometric excess observed in the F410M band (dashed oval). Overall, JADES-GS-z7-LA has a `bluer' SED, which is reflected in the preference for a bursty star-formation history when compared to the companion source. (*fig:SED*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

214  publications in the last 7 days.
	 _build/html/2302.12805.md
	 _build/html/2302.10943.md
	 _build/html/2302.10528.md
	 _build/html/2302.10008.md
	 _build/html/2302.08962.md
	 _build/html/2302.08628.md
	 _build/html/2302.07916.md
	 _build/html/2302.07880.md
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> daily </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


# Debugging papers